<h3>Build up the network</h3>
Your Genesis cloud API token needs to be set in the environment variable GC_API_TOKEN, the name of your SSH key should be set in the variable GC_SSH_KEY_NAME. If you are sure that you won't accidently push them to github, you can also overwrite them in here.

In [1]:
import os
from create_nodes import *

In [4]:
ssh_key = os.getenv('GC_SSH_KEY_NAME')
api_token = os.getenv('GC_API_TOKEN')

In [6]:
my_client = Client(api_token)
if not my_client.connect():
    exit()

In [8]:
gateway = create_gateway(my_client, ssh_key)

Using the prebuild snapshot with docker installed.
Instance 42ae68f8-acc2-4df3-b370-bb92a8039a29 is starting.
instance is active at 194.61.21.198


In [9]:
create_workers(my_client, ssh_key, gateway.private_ip)

Launching Alice as a worker node on a separate instance.
Worker Alice(4ea37418-7cf3-491e-93e2-b180d82724d6) is starting.


[Instance({'id': '4ea37418-7cf3-491e-93e2-b180d82724d6', 'name': 'Alice', 'hostname': 'Alice', 'type': 'vcpu-4_memory-12g_disk-80g_nvidia1080ti-1', 'allowed_actions': ['start', 'shutoff', 'reset'], 'ssh_keys': [{'id': '98427258-109f-47be-8b27-20bf9ffb86d9', 'name': 'laptop-key'}], 'image': {'id': '4ebc5a6e-7e27-4ab1-8b36-09ec535e047e', 'name': 'nvidia+docker'}, 'security_groups': [{'id': '6ff66fd5-1a04-4e85-9dfe-99c8c98da73f', 'name': 'standard'}, {'id': '865e6bf8-f04d-4d94-bfeb-d3911c7d90c2', 'name': 'pygrid'}], 'status': 'active', 'private_ip': '192.168.10.120', 'public_ip': '194.61.20.203', 'created_at': '2020-06-03T17:13:36.720Z', 'updated_at': '2020-06-03T17:14:28.901Z'})]

<h3>Import all the required pysyft</h3>

In [1]:
import torch
import syft as sy
from syft.grid.public_grid import PublicGridNetwork
from syft.workers.node_client import NodeClient
hook = sy.TorchHook(torch)
sy.hook.local_worker.is_client_worker = False

In [4]:
public_grid = PublicGridNetwork(hook, "http://"+gateway_instance['ip']+":5000")
brutus = NodeClient(hook,"http://"+gateway_instance['ip']+":3000")
alice = NodeClient(hook,"http://"+nodes[0].ip+":3000")

In [7]:
a = torch.tensor([1,2,3])
a_ptr = a.send(alice)
a_ptr = a.send(brutus)
a_ptr.get()

tensor([1, 2, 3])